In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

In [21]:
ins = pd.read_csv('../../data/installments_payments.csv')

결측치 확인 및 처리

In [22]:
ins.isnull().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

In [3]:
# ins = ins[ins['DAYS_ENTRY_PAYMENT'].notnull()]

In [5]:
# NUM_INSTALMENT_VERSION 피쳐가 의미가 있는지에 대한 고민

# ins.drop('NUM_INSTALMENT_VERSION', axis=1, inplace=True)

피쳐와 피쳐를 결합하여 새로운 피쳐 생성

In [25]:
# 납입일과 실제 납입일 차이
ins['DAYS_INADVANCE'] = (ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT'])

# 할부금과 실제 납입금 차이
ins['AMT_GAP'] = (ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT'])

In [27]:
ins

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DAYS_INADVANCE,AMT_GAP
512588,1000001,158271,1.0,1,-268.0,-294.0,6404.310,6404.310,26.0,0.0
2159480,1000001,158271,2.0,2,-238.0,-244.0,62039.115,62039.115,6.0,0.0
1214732,1000002,101962,1.0,1,-1600.0,-1611.0,6264.000,6264.000,11.0,0.0
1631862,1000002,101962,1.0,2,-1570.0,-1575.0,6264.000,6264.000,5.0,0.0
3411021,1000002,101962,1.0,3,-1540.0,-1559.0,6264.000,6264.000,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...
9623305,2843499,314148,1.0,9,-963.0,-980.0,16074.000,16074.000,17.0,0.0
10430159,2843499,314148,1.0,5,-1083.0,-1109.0,16074.000,16074.000,26.0,0.0
11071524,2843499,314148,1.0,6,-1053.0,-1074.0,16074.000,16074.000,21.0,0.0
11350339,2843499,314148,1.0,3,-1143.0,-1172.0,16074.000,16074.000,29.0,0.0


수치형 피쳐들에 대해 각각 groupby시 다른 agg 조건 설정

In [28]:
aggregation = {'NUM_INSTALMENT_NUMBER': ['max', 'mean'], 'DAYS_INADVANCE': ['mean'], 'NUM_INSTALMENT_VERSION': ['max', 'min'],
              'AMT_INSTALMENT':['mean'], 'AMT_PAYMENT':['mean'],'AMT_GAP': ['max', 'min', 'mean']}
aggregation

{'NUM_INSTALMENT_NUMBER': ['max', 'mean'],
 'DAYS_INADVANCE': ['mean'],
 'NUM_INSTALMENT_VERSION': ['max', 'min'],
 'AMT_INSTALMENT': ['mean'],
 'AMT_PAYMENT': ['mean'],
 'AMT_GAP': ['max', 'min', 'mean']}

위에서 설정한 조건에 따라 SK_ID_PREV를 기준으로 그룹화

In [29]:
ins = ins.groupby(['SK_ID_PREV', 'SK_ID_CURR']).agg(aggregation)

In [31]:
# 컬럼 구분 및 이름 변경
ins.columns = pd.Index([e[0] + "_" + e[1].upper() for e in ins.columns.tolist()])

In [33]:
# 확인
ins

,,NUM_INSTALMENT_NUMBER_MAX,NUM_INSTALMENT_NUMBER_MEAN,DAYS_INADVANCE_MEAN,NUM_INSTALMENT_VERSION_MAX,NUM_INSTALMENT_VERSION_MIN,AMT_INSTALMENT_MEAN,AMT_PAYMENT_MEAN,AMT_GAP_MAX,AMT_GAP_MIN,AMT_GAP_MEAN
SK_ID_PREV,SK_ID_CURR,,,,,,,,,,
1000001,158271,2,1.500000,16.000000,2.0,1.0,34221.712500,34221.712500,0.000,0.0,0.000000
1000002,101962,4,2.500000,19.750000,2.0,1.0,9308.891250,9308.891250,0.000,0.0,0.000000
1000003,252457,3,2.000000,15.333333,1.0,1.0,4951.350000,4951.350000,0.000,0.0,0.000000
1000004,260094,7,4.000000,26.714286,2.0,1.0,4789.022143,4789.022143,0.000,0.0,0.000000
1000005,176456,10,5.818182,8.454545,1.0,1.0,14703.210000,13365.609545,14710.815,0.0,1337.600455
...,...,...,...,...,...,...,...,...,...,...,...
2843495,260963,7,4.000000,3.857143,2.0,1.0,113932.883571,113932.883571,0.000,0.0,0.000000
2843496,425374,32,16.235294,4.000000,0.0,0.0,9186.311912,8950.975147,4664.700,0.0,235.336765
2843497,451578,20,10.500000,2.900000,1.0,1.0,9175.185000,9175.185000,0.000,0.0,0.000000


결측치 재확인

In [34]:
ins.isnull().sum()

NUM_INSTALMENT_NUMBER_MAX      0
NUM_INSTALMENT_NUMBER_MEAN     0
DAYS_INADVANCE_MEAN           78
NUM_INSTALMENT_VERSION_MAX     0
NUM_INSTALMENT_VERSION_MIN     0
AMT_INSTALMENT_MEAN            0
AMT_PAYMENT_MEAN              78
AMT_GAP_MAX                   78
AMT_GAP_MIN                   78
AMT_GAP_MEAN                  78
dtype: int64

SK_ID_CURR를 기준으로 그룹화

이전에 한번 그룹화 되었던 데이터프레임이기 때문에 다시 한 번 그룹화 할 때는 평균으로 agg 기준 설정 및 그룹화

In [35]:
ins = ins.groupby('SK_ID_CURR').agg(['mean'])

In [36]:
ins

,NUM_INSTALMENT_NUMBER_MAX,NUM_INSTALMENT_NUMBER_MEAN,DAYS_INADVANCE_MEAN,NUM_INSTALMENT_VERSION_MAX,NUM_INSTALMENT_VERSION_MIN,AMT_INSTALMENT_MEAN,AMT_PAYMENT_MEAN,AMT_GAP_MAX,AMT_GAP_MIN,AMT_GAP_MEAN
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
SK_ID_CURR,,,,,,,,,,
100001,4.000000,2.750000,5.916667,1.500000,1.0,5647.200000,5647.200000,0.0000,0.0000,0.000000
100002,19.000000,10.000000,20.421053,2.000000,1.0,11559.247105,11559.247105,0.0000,0.0000,0.000000
100003,8.333333,4.666667,7.448413,1.333333,1.0,78558.479286,78558.479286,0.0000,0.0000,0.000000
100004,3.000000,2.000000,7.666667,2.000000,1.0,7096.155000,7096.155000,0.0000,0.0000,0.000000
100005,9.000000,5.000000,23.555556,2.000000,1.0,6240.205000,6240.205000,0.0000,0.0000,0.000000
...,...,...,...,...,...,...,...,...,...,...
456251,7.000000,4.000000,36.285714,2.000000,1.0,7492.924286,7492.924286,0.0000,0.0000,0.000000
456252,6.000000,3.500000,2.833333,1.000000,1.0,10069.867500,10069.867500,0.0000,0.0000,0.000000


In [37]:
# 컬럼 구분 및 이름 변경
ins.columns = pd.Index(['INS_' + e[0] + "_" + e[1].upper() for e in ins.columns.tolist()])

In [38]:
# 확인
ins

,INS_NUM_INSTALMENT_NUMBER_MAX_MEAN,INS_NUM_INSTALMENT_NUMBER_MEAN_MEAN,INS_DAYS_INADVANCE_MEAN_MEAN,INS_NUM_INSTALMENT_VERSION_MAX_MEAN,INS_NUM_INSTALMENT_VERSION_MIN_MEAN,INS_AMT_INSTALMENT_MEAN_MEAN,INS_AMT_PAYMENT_MEAN_MEAN,INS_AMT_GAP_MAX_MEAN,INS_AMT_GAP_MIN_MEAN,INS_AMT_GAP_MEAN_MEAN
SK_ID_CURR,,,,,,,,,,
100001,4.000000,2.750000,5.916667,1.500000,1.0,5647.200000,5647.200000,0.0000,0.0000,0.000000
100002,19.000000,10.000000,20.421053,2.000000,1.0,11559.247105,11559.247105,0.0000,0.0000,0.000000
100003,8.333333,4.666667,7.448413,1.333333,1.0,78558.479286,78558.479286,0.0000,0.0000,0.000000
100004,3.000000,2.000000,7.666667,2.000000,1.0,7096.155000,7096.155000,0.0000,0.0000,0.000000
100005,9.000000,5.000000,23.555556,2.000000,1.0,6240.205000,6240.205000,0.0000,0.0000,0.000000
...,...,...,...,...,...,...,...,...,...,...
456251,7.000000,4.000000,36.285714,2.000000,1.0,7492.924286,7492.924286,0.0000,0.0000,0.000000
456252,6.000000,3.500000,2.833333,1.000000,1.0,10069.867500,10069.867500,0.0000,0.0000,0.000000
456253,7.333333,5.700000,11.977778,1.000000,1.0,4104.814000,3839.941000,1315.2750,0.0000,264.873000


지금까지의 작업으로 만들어진 데이터프레임을 joblib 파일로 저장

In [41]:
joblib.dump(filename = 'ins_df.joblib', value=ins)

['ins_df.joblib']